In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
cluster_name = "UdacityStandardD2V2"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
# try:
#     compute_cluster = ComputeTarget(workspace=ws,name=cluster_name)
#     print("cluster exists")
# except ComputeTargetException:
amlcompute_Config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_v2",max_nodes=4)
compute_cluster = ComputeTarget.create(workspace=ws,name=cluster_name,provisioning_configuration=amlcompute_Config)
compute_cluster.wait_for_completion(show_output=True)


InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [7]:
from azureml.widgets import RunDetails
# from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--max_iter': choice(10,20,30,40,50,75,100),
    '--C': uniform(0.0,2.0)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=3,slack_factor=0.5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='./training',script='train.py',compute_target=compute_cluster,environment=sklearn_env,)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,hyperparameter_sampling=ps,policy=policy,primary_metric_name='Accuracy',
                                        primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,max_total_runs=10,max_concurrent_runs=2)

In [10]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
experiment = Experiment(workspace=ws, name='udacity_project1')
run = experiment.submit(config=hyperdrive_config)

# Show the status in the notebook as the experiment runs
# RunDetails(run).show()
# run.wait_for_completion()

In [11]:
run.wait_for_completion()

{'runId': 'HD_9fe6f251-a8e3-4e5f-8bf3-73b2634883e4',
 'target': 'UdacityStandardD2V2',
 'status': 'Completed',
 'startTimeUtc': '2023-08-23T06:15:05.108467Z',
 'endTimeUtc': '2023-08-23T06:22:43.563489Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '19f2aec2-d7d4-4ddd-9457-40d8fa0f501d',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1035-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.49.0',
  'space_size': 'infinite_space_size',
  'score': '0.9073804321437242',
  'best_child_run_id': 'HD_9fe6f251-a8e3-4e5f-8bf3-73b2634883e4_4',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_9fe6f251-a8e3-4e5f-8bf3-73b2634883e4_4'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telem

In [13]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
best_run_details = best_run.get_details()
print('Best Run Id: ', best_run.id)
print(' -Accuracy:', best_run_metrics['Accuracy'])
print(' -Arguments:',best_run_details)

Best Run Id:  HD_9fe6f251-a8e3-4e5f-8bf3-73b2634883e4_4
 -Accuracy: 0.9073804321437242
 -Arguments: {'runId': 'HD_9fe6f251-a8e3-4e5f-8bf3-73b2634883e4_4', 'target': 'UdacityStandardD2V2', 'status': 'Completed', 'startTimeUtc': '2023-08-23T06:18:15.887786Z', 'endTimeUtc': '2023-08-23T06:18:48.370332Z', 'services': {}, 'properties': {'_azureml.ComputeTargetType': 'amlctrain', 'ContentSnapshotId': '19f2aec2-d7d4-4ddd-9457-40d8fa0f501d', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath': False, 'arguments': ['--C', '0.920063757482074', '--max_iter', '50'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'UdacityStandardD2V2', 'dataReferences': {}, 'data': {}, 'outputData': {}, 'datacaches': [], 'jobName': None, 'maxRunDurationSeconds': 2592000, 'nodeCount': 1, 'instanceType

In [14]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data = TabularDatasetFactory().from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [17]:
from training.train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(data)

ModuleNotFoundError: No module named 'training'

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###